In [454]:
import numpy as np
import pandas as pd
import pandas_ta as ta
import datetime
from tqdm import tqdm
tqdm.pandas()
import csv
def setup():
  df  = pd.read_csv('../Datasets/Binance_BTCUSDT_1h.csv', header=1, usecols=['unix','Volume BTC','high','low','close', 'open'], nrows=1680)
  print(len(df))
  df.dropna(inplace=True, how='any')
  #'open', 'high','low', 'Volume BTC'
  #Transform unix into date readable
  df['date']=pd.to_datetime(df['unix'],unit='ms')
  df.drop(columns='unix', inplace=True)

  df.set_index('date', inplace=True)

  df['diff'] = df['close'] - df['open']

  def simplifier(val):
    if(val > 50):
      return 2
    elif(val<-50):
      return 0
    return 1

  df['y'] = df['diff'].progress_apply(simplifier)
  df = df[::-1]
  df['y']=df['y'].shift(-1)
  testValue = df.iloc[-34:]
  df = df[:-1]

  df.drop(columns=['open','diff'], inplace=True)
  return df


# Machine Learning

In [462]:
df = setup()

1680


100%|██████████| 1680/1680 [00:00<00:00, 566615.53it/s]


In [463]:
len(df)

1679

In [464]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import pandas_ta as ta

In [465]:
strat1 = ta.Strategy(name='Strategie 1',
                     description='MACD, RSI',
                     ta=[
                      {'kind':'macd', 'fast':12 , 'slow':26},
                      {'kind':'rsi', 'length':14},
                      {'kind':'rsi', 'length':7},
                      {'kind':'obv'},
                      {'kind':'sma', 'length':99},
                      {'kind':'sma', 'length':50},
                      {'kind':'sma', 'length':200},
                      {'kind':'stoch'},
                      {'kind':'rsi', 'length':21},
                      {'kind':'rsi', 'length':28},
                      {'kind':'ppo'},
                      {'kind':'psar'},
                      {'kind':'adx'},
                      {'kind':'bbands'},
                      
                     ])

In [466]:
#Apply the strategy
df.ta.strategy(strat1)
#Remove first value because of nan
df.drop(columns='PSARs_0.02_0.2', inplace=True)


# 


In [460]:
# df.dropna(inplace=True, how='any')

In [468]:
df.head(200)

,high,low,close,Volume BTC,y,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,RSI_14,RSI_7,...,PSARaf_0.02_0.2,PSARr_0.02_0.2,ADX_14,DMP_14,DMN_14,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
date,,,,,,,,,,,,,,,,,,,,,
2022-05-20 01:00:00,30777.33,30130.48,30475.57,4218.55549,0.0,NaN,NaN,NaN,NaN,NaN,...,0.02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-20 02:00:00,30512.80,30228.63,30336.34,1661.00444,0.0,NaN,NaN,NaN,NaN,NaN,...,0.02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-20 03:00:00,30345.24,30136.08,30175.03,1631.13443,0.0,NaN,NaN,NaN,NaN,NaN,...,0.02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-20 04:00:00,30249.10,30012.00,30099.67,1621.85469,2.0,NaN,NaN,NaN,NaN,NaN,...,0.02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-20 05:00:00,30194.27,30025.45,30185.39,1258.50356,0.0,NaN,NaN,NaN,NaN,NaN,...,0.02,0,NaN,NaN,NaN,29985.128862,30254.400,30523.671138,1.780046,0.371858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-28 04:00:00,28890.00,28754.40,28833.13,1362.65989,1.0,-98.958465,27.083814,-126.042279,48.887225,53.736092,...,0.04,0,24.889220,16.741952,20.381068,28580.140546,28752.080,28924.019454,1.196014,0.735693
2022-05-28 05:00:00,28949.39,28777.02,28876.96,1812.58195,1.0,-82.959664,34.466091,-117.425756,50.043147,56.128181,...,0.06,0,23.502257,17.492439,19.517536,28569.010360,28766.212,28963.413640,1.371064,0.780799
2022-05-28 06:00:00,28917.98,28770.78,28865.21,1267.23219,1.0,-70.416904,37.607081,-108.023986,49.718532,55.234953,...,0.06,0,22.244364,16.836398,18.944531,28679.402372,28816.090,28952.777628,0.948690,0.679680


In [390]:
# from sklearn.model_selection import cross_val_score


def modelForest(col):
  X = df[col]
  y = df['y']
  X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.9, )
  
  rfc = RandomForestClassifier(bootstrap=True, oob_score=True,random_state=42, n_estimators=100,)
  
  param_grid = { 
        'max_depth' : [3,5,10, 20, None],
        'min_samples_leaf' : [1,3,5],
        'max_features' : ['sqrt',0.5]
          
    } 
  
  CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=2, n_jobs=-1,  error_score='raise')
  CV_rfc.fit(X_train, y_train)
  
  return CV_rfc.best_estimator_.oob_score_ , CV_rfc.best_params_, CV_rfc.score(X_test,y_test)
  

In [391]:
import itertools
l = []
a = list(df.columns)
# a.remove('PSARs_0.02_0.2')
a.remove('low')
a.remove('close')
a.remove('high')
a.remove('Volume BTC')
b = [x for x in a if 'y' not in x]

for i in range(2,5):
   l.append(list(itertools.combinations(b,i)))

l2 = [item for sublist in l for item in sublist][1:]
maxOOB = 0
maxUnseen = 0


listOOB =[]
listUnseen = []

for l3 in tqdm(l2):
   res = modelForest(list(l3))
   n = res[0]
   m= res[1]
   p = res[2]
   
   
   if(n>maxOOB):
      maxOOB = n
      listOOB.append('Oob:' + str(max) +' -- '+ 'Unseen: '+str(p) +' -- '+ str(l3) + ' ---- ' + str(m))
      print('Oob:' + str(max) +' -- '+ 'Unseen: '+str(p) +' -- '+ str(l3) + ' ---- ' + str(m))
   if(p>maxUnseen):
      maxUnseen = p
      listUnseen.append('Unseen: '+str(maxUnseen) + ' -- ' + 'Oob: ' + str(n) +' -- '+ str(l3) + ' ---- ' + str(m))
      # print('Unseen: '+str(p))



  0%|          | 1/20825 [00:04<27:42:44,  4.79s/it]

Oob:0.47518796992481205 -- Unseen: 0.5064935064935064 -- ('MACD_12_26_9', 'MACDs_12_26_9') ---- {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1}


  0%|          | 4/20825 [00:10<12:33:30,  2.17s/it]

Oob:0.47518796992481205 -- Unseen: 0.4935064935064935 -- ('MACD_12_26_9', 'OBV') ---- {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1}


  0%|          | 32/20825 [01:03<11:29:39,  1.99s/it]

...

  1%|          | 257/20825 [09:22<13:17:39,  2.33s/it]

Oob:0.47518796992481205 -- Unseen: 0.4155844155844156 -- ('RSI_28', 'BBU_5_2.0') ---- {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1}


  9%|▊         | 1784/20825 [1:05:24<11:42:43,  2.21s/it]

Oob:0.47518796992481205 -- Unseen: 0.38961038961038963 -- ('OBV', 'SMA_200', 'PSARr_0.02_0.2') ---- {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 5}


100%|██████████| 20825/20825 [13:14:16<00:00,  2.29s/it]  


In [164]:
# ('OBV', 'SMA_25', 'SMA_99') -- 0.5506329113924051 ---- {'max_features': 'sqrt', 'n_estimators': 1700}
# 'SMA_50', 'RSI_10', 'RSI_24') -- 0.5180451127819549 ---- {'max_depth': 5, 'n_estimators': 2100}

In [393]:
listOOB
# "Unseen: 0.6753246753246753 -- Oob: 0.4444444444444444 -- ('MACD_12_26_9', 'SMA_200', 'PPOh_12_26_9', 'BBP_5_2.0') ---- {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 3}"

["Oob:0.47518796992481205 -- Unseen: 0.5064935064935064 -- ('MACD_12_26_9', 'MACDs_12_26_9') ---- {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1}",
 "Oob:0.47518796992481205 -- Unseen: 0.4935064935064935 -- ('MACD_12_26_9', 'OBV') ---- {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1}",
 "Oob:0.47518796992481205 -- Unseen: 0.35064935064935066 -- ('MACDh_12_26_9', 'SMA_200') ---- {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 1}",
 "Oob:0.47518796992481205 -- Unseen: 0.4025974025974026 -- ('MACDs_12_26_9', 'PSARl_0.02_0.2') ---- {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 3}",
 "Oob:0.47518796992481205 -- Unseen: 0.37662337662337664 -- ('SMA_200', 'BBU_5_2.0') ---- {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 5}",
 "Oob:0.47518796992481205 -- Unseen: 0.4155844155844156 -- ('RSI_28', 'BBL_5_2.0') ---- {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 3}",
 "Oob:0.47518796992481205 -- Unseen: 0.4155844

In [433]:
from sklearn.model_selection import cross_val_score


X = df[['MACD_12_26_9', 'SMA_200', 'PPOh_12_26_9', 'BBP_5_2.0']]
y = df['y']
l1 = []
l2 = []
# for i in range(4):
X_train, X_test, y_train, y_test = train_test_split(X,y)
rfc = RandomForestClassifier(max_depth= 3, max_features= 'sqrt', min_samples_leaf= 3 ,random_state=42)
l1.append(np.mean(cross_val_score(rfc, X_train, y_train, cv=16, n_jobs=-1)))
rfc.fit(X_train,y_train)
l2.append(rfc.score(X_test, y_test))
print('CrossVal: ' + str(np.mean(l1)))
print('Unseen: ' + str(np.mean(l2)))



CrossVal: 0.472972972972973
Unseen: 0.49222797927461137


DecisionTreeClassifier()

In [85]:
import pickle
#Save model
filename='model.pkl'
with open(filename, 'wb') as file:
  pickle.dump(None, file) ##############